## LCEL (LangChain Expression Language)

LCEL เป็น “ภาษาเชิงประกาศ (declarative)” สำหรับประกอบ chains จากหน่วยพื้นฐานที่เรียกว่า Runnable (เช่น Prompt, LLM, Parser, Retriever) โดยใช้ตัวต่อแบบท่อ | เพื่อบอก “อยากให้เกิดอะไร” มากกว่าบอกขั้นตอน imperatively

- Pipe operator (|) ต่อ runnable เป็น RunnableSequence: เอาเอาต์พุตของตัวก่อนหน้าไปเป็นอินพุตของตัวถัดไป (ทำ pipeline)

- RunnableParallel รันหลายงานพร้อมกัน แล้วรวมผลเป็น dict (เช่น ดึง context กับคำถามพร้อม ๆ กันสำหรับ RAG)

- Retriever เป็น Runnable โดยดีไซน์ (สืบทอดจาก BaseRetriever) จึงต่อเข้าพายป์ไลน์ LCEL ได้ตรง ๆ ไม่ต้องฝังในพรอมป์ตเสมอไป

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

### Basic Chain (LCEL/Runnable) 

แสดง pipeline prompt → llm → parser

In [6]:
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
llm = ChatOllama(model="gpt-oss")
parser = StrOutputParser()

chain = prompt | llm | parser
print(chain.invoke({"topic": "ice cream"}))

Why did the ice cream go to therapy?  
Because it couldn’t stop melting under pressure!


### Chain (LCEL/Runnable) -> RAG

“ดึง context และส่ง question” พร้อมกัน แล้วค่อยประกอบเป็นพรอมป์ต → LLM

In [7]:
from langchain_core.tools import tool

@tool
def calculate_math_tool(expression: str):
    """Calculate mathematical expressions safely"""
    try:
        # Remove any potentially dangerous characters and evaluate safely
        allowed_chars = set('0123456789+-*/(). ')
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return f"ผลลัพธ์: {expression} = {result}"
        else:
            return "ข้อผิดพลาด: นิพจน์ไม่ถูกต้อง ใช้ได้เฉพาะตัวเลขและเครื่องหมาย +, -, *, /, (, ) เท่านั้น"
    except Exception as e:
        return f"ข้อผิดพลาดในการคำนวณ: {str(e)}"

calculate_math_tool.invoke("1+2")

'ผลลัพธ์: 1+2 = 3'

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

from langchain.agents import Tool

serper = GoogleSerperAPIWrapper()

search_tool = Tool(
    name="search",
    description="Useful for when you need more information from an online search",
    func=serper.run
)

LCEL เน้น declarative และ parallel/batch/stream ได้ง่าย เหมาะกับ workflow ตายตัว (เช่น RAG, summarize, ETL) มากกว่า “ให้โมเดลเลือก tool เอง”

In [9]:
# from langchain_core.runnables import RunnableParallel, RunnableSequence
from operator import itemgetter

prompt = ChatPromptTemplate.from_template(
    "สรุปจากผลค้นหา:\n{search_out}\n\n"
    "และผลคำนวณ:\n{calc_out}\n\n"
    "จากนั้นตอบให้ชัดเจน:"
)

parallel = {
    "search_out": itemgetter("q") | search_tool,
    "calc_out": itemgetter("expr") | calculate_math_tool
}
chain = parallel | prompt | llm

# chain = RunnableSequence([
#     RunnableParallel({
#         "search_out": itemgetter("q")   | search_tool,
#         "calc_out":   itemgetter("expr")| calculate_math_tool,
#     }),
#     prompt,
#     llm,
# ])

# print(rag_chain.invoke("1+9 ได้เท่าไหร่"))

res = chain.invoke({
    "q": "สรุปข่าว AMD วันนี้สั้น ๆ",
    "expr": "(6+2)*3"
})

print(res)
print(res.content)

content='**สรุปข่าวสารล่าสุดเกี่ยวกับหุ้น AMD**\n\n| หัวข้อ | สรุป |\n|--------|------|\n| **ศักยภาพเพิ่มขึ้น 3 เท่า** | Mizuho รายงานว่าตลาดชิป AMD มีโอกาสเติบโตสูงถึงสามเท่าในอนาคต |\n| **ความร่วมมือกับ IBM** | AMD และ IBM กำลังสำรวจวิธีผสานรวมโปรเซสเซอร์, กราฟิกการ์ด และ FPGA ของ AMD เข้ากับเทคโนโลยีควอนตัมของ IBM เพื่อเร่งพัฒนาอัลกอริทึมใหม่ ๆ |\n| **การขายหุ้นของ CTO** | Mark Papermaster ของ AMD ขายหุ้นมูลค่า 1.99 ล้านดอลลาร์ |\n| **ผลประกอบการไตรมาส 2 ปี 2025** | รายได้ 7.69\u202fพันล้านดอลลาร์ (เกินอ้างประมาณการ 7.43\u202fพันล้านดอลลาร์) แต่หุ้นร่วงเกือบ 4\u202f% เนื่องจากปัจจัยอื่น ๆ |\n| **ความร่วมมือกับ Intel** | Intel ได้จัดตั้งกลุ่ม Big Tech ร่วมมือกับ AMD เพื่อพัฒนาชิป AI สำหรับ PC |\n| **ผลกระทบจากการขาดแคลนชิปรุ่นใหม่** | ความต้องการชิป GPU ที่สูงจาก Nvidia ทำให้ AMD มีความท้าทายในการส่งมอบผลิตภัณฑ์ |\n| **การปรับเปลี่ยนมุมมองของนักวิเคราะห์** | GF Securities (ฮ่องกง) ลดมุมมองเกี่ยวกับ AMD หลังการลดคำสั่งซื้อ MI355X ซึ่งทำให้ยอดออเดอร์ทั้งปีลดลง |\n| **ผลประกอบการล่าสุด*